# Análisis Exploratorio de Datos - Demanda Hotelera en Mendoza
## Segunda Entrega - Predicción de Demanda de Turismo Internacional

**Objetivo:** Realizar un análisis exploratorio detallado del dataset generado para predecir la demanda hotelera de turistas no residentes en Mendoza, Córdoba y Buenos Aires.

**Variables principales:**
- Turistas no residentes que ingresan a Mendoza
- Índice de interés en Google Trends
- Tipo de cambio del dólar (histórico)
- Estaciones del año y eventos anuales
- Conectividad aérea internacional

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Carga y Preparación de Datos

In [ ]:
# Cargar datasets (ajusta los paths según tu estructura)
try:
    df_turistas = pd.read_csv('data/turistas_no_residentes_mendoza.csv')
    df_dolar = pd.read_csv('data/dolar_historico.csv')
    df_trends = pd.read_csv('data/google_trends_mendoza.csv')
    print("✅ Datasets cargados desde archivos")
except:
    print("⚠️ No se encontraron archivos, generando datos de ejemplo...")
    dates = pd.date_range('2018-01-01', '2024-12-31', freq='M')
    np.random.seed(42)
    base = 12000; seasonal = 3000 * np.sin(np.arange(len(dates)) * 2 * np.pi / 12 - np.pi/2)
    trend = np.linspace(0, 2000, len(dates)); noise = np.random.normal(0, 800, len(dates))
    df_turistas = pd.DataFrame({'fecha': dates, 'turistas_no_residentes': np.maximum(base + seasonal + trend + noise, 5000)})
    df_dolar = pd.DataFrame({'fecha': dates, 'cotizacion_oficial': np.cumsum(np.random.normal(2, 8, len(dates))) + 80})
    trends_base = 40 + 0.002 * df_turistas['turistas_no_residentes'] + np.random.normal(0, 10, len(dates))
    df_trends = pd.DataFrame({'fecha': dates, 'tendencia_mendoza': np.clip(trends_base, 0, 100)})

In [ ]:
# Unificación de datasets
for df in [df_turistas, df_dolar, df_trends]:
    df['fecha'] = pd.to_datetime(df['fecha'])
df_main = df_turistas.merge(df_dolar, on='fecha').merge(df_trends, on='fecha')
# Variables temporales
df_main['año'] = df_main['fecha'].dt.year
df_main['mes'] = df_main['fecha'].dt.month
df_main['trimestre'] = df_main['fecha'].dt.quarter
df_main['estacion'] = df_main['mes'].map({12:'Verano',1:'Verano',2:'Verano',3:'Otoño',4:'Otoño',5:'Otoño',6:'Invierno',7:'Invierno',8:'Invierno',9:'Primavera',10:'Primavera',11:'Primavera'})
df_main['temporada_vendimia'] = ((df_main['mes']>=2)&(df_main['mes']<=4)).astype(int)
df_main['temporada_ski'] = ((df_main['mes']>=6)&(df_main['mes']<=9)).astype(int)
print(df_main.head())

## 2. Estadísticas Descriptivas

In [ ]:
print(df_main.describe().round(2))
print('\nValores faltantes:')
print(df_main.isnull().sum())

## 3. Análisis Univariado

In [ ]:
variables = ['turistas_no_residentes','cotizacion_oficial','tendencia_mendoza']
fig, axes = plt.subplots(1,3,figsize=(18,5))
for i,var in enumerate(variables):
    sns.histplot(df_main[var], bins=20, ax=axes[i], kde=True)
    axes[i].set_title(f'Distribución de {var.replace("_"," ").title()}')
plt.tight_layout(); plt.show()

## 4. Análisis Bivariado y Correlación

In [ ]:
corr = df_main[['turistas_no_residentes','cotizacion_oficial','tendencia_mendoza']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()

sns.scatterplot(x='cotizacion_oficial', y='turistas_no_residentes', data=df_main)
plt.title('Turistas vs Dólar oficial')
plt.show()

sns.scatterplot(x='tendencia_mendoza', y='turistas_no_residentes', data=df_main)
plt.title('Turistas vs Google Trends')
plt.show()

## 5. Análisis Temporal y Estacionalidad

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
ax.plot(df_main['fecha'], df_main['turistas_no_residentes'], label='Turistas no residentes')
ax.set_title('Evolución de Turistas no Residentes en Mendoza')
ax.set_ylabel('Cantidad')
plt.legend(); plt.show()

df_main.groupby('mes')['turistas_no_residentes'].mean().plot(kind='bar', color='skyblue')
plt.title('Promedio de Turistas por Mes')
plt.ylabel('Turistas no residentes'); plt.xlabel('Mes'); plt.show()

## 6. Detección de Outliers

In [ ]:
q1 = df_main['turistas_no_residentes'].quantile(0.25)
q3 = df_main['turistas_no_residentes'].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
outliers = df_main[(df_main['turistas_no_residentes'] < lower) | (df_main['turistas_no_residentes'] > upper)]
print(f'Cantidad de outliers detectados: {len(outliers)}')
sns.boxplot(x=df_main['turistas_no_residentes']); plt.title('Boxplot Turistas no residentes'); plt.show()

## 7. Hipótesis y Hallazgos Preliminares

- La demanda hotelera internacional en Mendoza exhibe marcada estacionalidad, con máximos en verano y época de vendimia.
- El tipo de cambio parece estar positivamente correlacionado con la llegada de turistas internacionales.
- El interés digital (Google Trends) anticipa la demanda real, lo que se evidencia en la correlación y en el comportamiento temporal.
- Las diferencias por eventos especiales (vendimia, ski) son notorias y estadísticamente significativas.
- Se detectan algunos outliers, habitualmente en picos o caídas abruptas (verificar si corresponden a eventos o errores de carga).

## 8. Variables Más Relevantes para el Modelo

In [ ]:
corr_abs = corr['turistas_no_residentes'].abs().sort_values(ascending=False)
print('Variables ordenadas por correlación absoluta:')
print(corr_abs)